In [20]:
import duckdb

years = range(2000, 2024)
urls = [f"'https://raw.githubusercontent.com/JeffSackmann/tennis_atp/refs/heads/master/atp_matches_{year}.csv'" for year in years]
concat_urls = ", ".join(urls)
stats = duckdb.query(f"select * from read_csv([{concat_urls}]) where w_svpt > 0 and l_svpt > 0").df()

stats

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_seed,winner_entry,...,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points
0,2000-301,Auckland,Hard,32,A,20000110,1,103163,1.0,None,...,55,39,29,17,4,7,11.0,1612.0,63.0,595.0
1,2000-301,Auckland,Hard,32,A,20000110,2,102607,NaN,Q,...,32,25,18,12,3,6,211.0,157.0,49.0,723.0
2,2000-301,Auckland,Hard,32,A,20000110,3,103252,NaN,None,...,33,20,7,8,7,11,48.0,726.0,59.0,649.0
3,2000-301,Auckland,Hard,32,A,20000110,4,103507,7.0,None,...,43,29,14,10,6,8,45.0,768.0,61.0,616.0
4,2000-301,Auckland,Hard,32,A,20000110,5,102103,NaN,Q,...,46,34,18,12,5,9,167.0,219.0,34.0,873.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65356,2022-M-DC-2022-WG2-PO-GRE-JAM-01,Davis Cup WG2 PO: GRE vs JAM,Clay,4,D,20220304,4,209362,NaN,None,...,68,42,12,10,11,13,1103.0,9.0,1130.0,8.0
65357,2022-M-DC-2022-WG2-PO-GRE-JAM-01,Davis Cup WG2 PO: GRE vs JAM,Clay,4,D,20220304,5,202065,NaN,None,...,56,40,20,15,4,8,808.0,23.0,1390.0,4.0
65358,2022-M-DC-2022-WG2-PO-HKG-BEN-01,Davis Cup WG2 PO: HKG vs BEN,Hard,4,D,20220304,1,138846,NaN,None,...,54,29,8,11,6,10,1059.0,10.0,1881.0,1.0
65359,2022-M-DC-2022-WG2-PO-HKG-BEN-01,Davis Cup WG2 PO: HKG vs BEN,Hard,4,D,20220304,2,209409,NaN,None,...,39,24,7,10,5,9,1050.0,10.0,NaN,NaN


In [21]:
clutch_stats = duckdb.query(
"""
with winner_centered as (
    select 
        winner_id as player_id, 
        winner_name as player_name,

        w_svpt as service_points,
        w_1stWon + w_2ndWon as service_points_won,

        l_svpt as return_points,
        l_svpt - (l_1stWon + l_2ndWon) as return_points_won,

        w_bpFaced as break_points_faced, 
        w_bpSaved as break_points_saved, 

        l_bpFaced as break_opportunities, 
        l_bpFaced - l_bpSaved as break_opportunities_converted
    from stats 
), loser_centered as (
    select 
        loser_id as player_id, 
        loser_name as player_name, 

        l_svpt as service_points,
        l_1stWon + l_2ndWon as service_points_won,

        w_svpt as return_points,
        w_svpt - (w_1stWon + w_2ndWon) as return_points_won,

        l_bpFaced as break_points_faced, 
        l_bpSaved as break_points_saved, 
        
        w_bpFaced as break_opportunities, 
        w_bpFaced - w_bpSaved as break_opportunities_converted
    from stats 
), combined as (
    select * from winner_centered
    union all
    select * from loser_centered
), actual_vs_expected as (
    select 
        *, 
        break_points_faced * service_points_won / service_points as expected_break_points_saved, 
        break_opportunities * return_points_won / return_points as expected_break_opportunities_converted
    from combined
), summarize_by_player as (
    select
        player_id,
        any_value(player_name) as player_name,
        sum(service_points) as service_points,
        sum(service_points_won) as service_points_won,
        sum(return_points) as return_points,
        sum(return_points_won) as return_points_won,
        sum(break_points_faced) as break_points_faced,
        sum(break_points_saved) as break_points_saved,
        sum(expected_break_points_saved) as expected_break_points_saved,
        sum(break_opportunities) as break_opportunities,
        sum(break_opportunities_converted) as break_opportunities_converted,
        sum(expected_break_opportunities_converted) as expected_break_opportunities_converted
    from actual_vs_expected
    group by player_id
), big3 as (
    select * from summarize_by_player where player_name in ('Roger Federer', 'Rafael Nadal', 'Novak Djokovic')
), all_players as (
    select 
        null as player_id,
        'All Players' as player_name,
        sum(service_points) as service_points,
        sum(service_points_won) as service_points_won,
        sum(return_points) as return_points,
        sum(return_points_won) as return_points_won,
        sum(break_points_faced) as break_points_faced,
        sum(break_points_saved) as break_points_saved,
        sum(expected_break_points_saved) as expected_break_points_saved,
        sum(break_opportunities) as break_opportunities,
        sum(break_opportunities_converted) as break_opportunities_converted,
        sum(expected_break_opportunities_converted) as expected_break_opportunities_converted
    from summarize_by_player
), big3_vs_all as (
    select * from big3
    union all
    select * from all_players
)
select 
    player_id,
    player_name,
    'Serve' as serve_or_return, 
    'Expected' as expected_or_actual,
    expected_break_points_saved / break_points_faced as value
from big3_vs_all
union all
select 
    player_id,
    player_name,
    'Serve' as serve_or_return, 
    'Actual' as expected_or_actual,
    break_points_saved / break_points_faced as value
from big3_vs_all
union all
select 
    player_id,
    player_name,
    'Return' as serve_or_return, 
    'Expected' as expected_or_actual,
    expected_break_opportunities_converted / break_opportunities as value
from big3_vs_all
union all
select 
    player_id,
    player_name,
    'Return' as serve_or_return, 
    'Actual' as expected_or_actual,
    break_opportunities_converted / break_opportunities as value
from big3_vs_all
"""
).df()
clutch_stats

,player_id,player_name,serve_or_return,expected_or_actual,value
0,104925.0,Novak Djokovic,Serve,Expected,0.642297
1,103819.0,Roger Federer,Serve,Expected,0.658700
2,104745.0,Rafael Nadal,Serve,Expected,0.634643
3,NaN,All Players,Serve,Expected,0.595872
4,104925.0,Novak Djokovic,Serve,Actual,0.655609
5,103819.0,Roger Federer,Serve,Actual,0.675550
6,104745.0,Rafael Nadal,Serve,Actual,0.661325
7,NaN,All Players,Serve,Actual,0.603426
8,104925.0,Novak Djokovic,Return,Expected,0.446856
9,103819.0,Roger Federer,Return,Expected,0.422843


In [22]:
import altair as alt

base = alt.Chart(clutch_stats).encode(
    x=alt.X('value', scale=alt.Scale(zero=False), title=None),
    y=alt.Y('player_name', title=None),
    color=alt.Color('serve_or_return', title=None),
)
circles = base.mark_point().encode(
    shape=alt.Shape('expected_or_actual', title=None)
)
lines = base.mark_line().encode(
    detail=['player_name', 'serve_or_return']
)
(circles + lines).properties(title='Expected vs Actual Break Point Win Rate')

alt.LayerChart(...)

In [23]:
clutch_stats = duckdb.query(
"""
with winner_centered as (
    select 
        winner_id as player_id, 
        winner_name as player_name,

        w_svpt as service_points,
        w_1stWon + w_2ndWon as service_points_won,

        l_svpt as return_points,
        l_svpt - (l_1stWon + l_2ndWon) as return_points_won,

        w_bpFaced as break_points_faced, 
        w_bpSaved as break_points_saved, 

        l_bpFaced as break_opportunities, 
        l_bpFaced - l_bpSaved as break_opportunities_converted
    from stats 
),
loser_centered as (
    select 
        loser_id as player_id, 
        loser_name as player_name, 

        l_svpt as service_points,
        l_1stWon + l_2ndWon as service_points_won,

        w_svpt as return_points,
        w_svpt - (w_1stWon + w_2ndWon) as return_points_won,

        l_bpFaced as break_points_faced, 
        l_bpSaved as break_points_saved, 
        
        w_bpFaced as break_opportunities, 
        w_bpFaced - w_bpSaved as break_opportunities_converted
    from stats 
),
combined as (
    select * from winner_centered
    union all
    select * from loser_centered
), player_level_stats as (
    select
        player_name,

        sum(break_points_saved) / sum(break_points_faced) as "Break Point Saved",
        sum(service_points_won) / sum(service_points) as "Service Points Won",

        sum(break_opportunities_converted) / sum(break_opportunities) as "Break Opportunities Converted",
        sum(return_points_won) / sum(return_points) as "Return Points Won",
    from combined
    group by all
), total_stats as (
    select
        'All Players' as player_name,

        sum(break_points_saved) / sum(break_points_faced) as "Break Point Saved",
        sum(service_points_won) / sum(service_points) as "Service Points Won",

        sum(break_opportunities_converted) / sum(break_opportunities) as "Break Opportunities Converted",
        sum(return_points_won) / sum(return_points) as "Return Points Won",
    from combined
    group by all
), unpivot_big_3 as (
    select * 
    from player_level_stats 
    unpivot (value for stat in ("Break Point Saved", "Service Points Won", "Break Opportunities Converted", "Return Points Won"))
    where player_name in ('Roger Federer', 'Rafael Nadal', 'Novak Djokovic')
), unpivot_total as (
    select * 
    from total_stats
    unpivot (value for stat in ("Break Point Saved", "Service Points Won", "Break Opportunities Converted", "Return Points Won"))
), combine_levels as (
    select * from unpivot_big_3
    union all
    select * from unpivot_total
)
select 
    *,
    case stat when 'Break Point Saved' then 'Serves'
        when 'Service Points Won' then 'Serves'
        when 'Break Opportunities Converted' then 'Returns'
        when 'Return Points Won' then 'Returns'
    end as is_serve,
    case stat when 'Break Point Saved' then 'Break Points'
        when 'Service Points Won' then 'All Points'
        when 'Break Opportunities Converted' then 'Break Points'
        when 'Return Points Won' then 'All Points'
    end as is_break_point,
from combine_levels
"""
).df()
clutch_stats

,player_name,stat,value,is_serve,is_break_point
0,Roger Federer,Break Point Saved,0.675550,Serves,Break Points
1,Roger Federer,Service Points Won,0.697212,Serves,All Points
2,Roger Federer,Break Opportunities Converted,0.411880,Returns,Break Points
3,Roger Federer,Return Points Won,0.397948,Returns,All Points
4,Rafael Nadal,Break Point Saved,0.661325,Serves,Break Points
5,Rafael Nadal,Service Points Won,0.673245,Serves,All Points
6,Rafael Nadal,Break Opportunities Converted,0.449080,Returns,Break Points
7,Rafael Nadal,Return Points Won,0.424039,Returns,All Points
8,Novak Djokovic,Break Point Saved,0.655609,Serves,Break Points
9,Novak Djokovic,Service Points Won,0.675611,Serves,All Points


In [24]:
import altair as alt

base = alt.Chart(clutch_stats).encode(
    x=alt.X('value', scale=alt.Scale(zero=False), title=None),
    y=alt.Y('player_name', title=None),
    color=alt.Color('is_serve', title=None),
)
circles = base.mark_point().encode(
    shape=alt.Shape('is_break_point', title=None)
)
lines = base.mark_line().encode(
    detail=['player_name', 'is_serve']
)
(circles + lines).properties(title='Break vs Non-Break Point Win Rate')

alt.LayerChart(...)